Install and load all dependencies (first time only) \
NOTE: you may need to restart the runtime afterwards (CTRL+M .).

In [1]:
pip install shimmy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 20.7 MB/s eta 0:00:00


In [2]:
!apt-get install -y \
    libgl1-mesa-dev \
    libgl1-mesa-glx \
    libglew-dev \
    libosmesa6-dev \
    software-properties-common

!apt-get install -y patchelf

!pip install gym
!pip install free-mujoco-py
!pip install stable-baselines3
!pip install cma

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
software-properties-common is already the newest version (0.99.22.9).
The following additional packages will be installed:
  libegl-dev libgl-dev libgles-dev libgles1 libglu1-mesa libglu1-mesa-dev libglvnd-core-dev
  libglvnd-dev libglx-dev libopengl-dev libosmesa6
The following NEW packages will be installed:
  libegl-dev libgl-dev libgl1-mesa-dev libgl1-mesa-glx libgles-dev libgles1 libglew-dev
  libglu1-mesa libglu1-mesa-dev libglvnd-core-dev libglvnd-dev libglx-dev libopengl-dev libosmesa6
  libosmesa6-dev
0 upgraded, 15 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,020 kB of archives.
After this operation, 19.4 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libglx-dev amd64 1.4.0-1 [14.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libgl-dev amd64 1.4.0-1 [101 kB]
Get:3 http://archive.ubuntu.com/ubuntu 

Set up the custom Hopper environment



1.   Upload `classes.zip` to the current session's file storage
2.   Un-zip it by running cell below


In [4]:
!unzip classes.zip

Archive:  classes.zip
   creating: env/
  inflating: __MACOSX/._env          
 extracting: env/__init__.py         
  inflating: __MACOSX/env/.___init__.py  
  inflating: env/custom_hopper.py    
  inflating: __MACOSX/env/._custom_hopper.py  
  inflating: env/mujoco_env.py       
  inflating: __MACOSX/env/._mujoco_env.py  
   creating: env/assets/
  inflating: __MACOSX/env/._assets   
  inflating: env/assets/hopper.xml   
  inflating: __MACOSX/env/assets/._hopper.xml  


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)




---



\

**Train an RL agent on the OpenAI Gym Hopper environment using REINFORCE and Actor-critic algorithms**

\


TASK 2 and 3: interleave data collection to policy updates

In [3]:
import gym
import os
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import CheckpointCallback, EvalCallback
from stable_baselines3.common.evaluation import evaluate_policy

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/util.py:55: DeprecationWarning: jax.xla_computation is deprecated. Please use the AOT APIs; see https://jax.readthedocs.io/en/latest/aot.html. For example, replace xla_computation(f)(*xs) with jit(f).lower(*xs).compiler_ir('hlo'). See CHANGELOG.md for 0.4.30 for more examples.
  from jax import xla_computation as _xla_computation


In [5]:
from env.custom_hopper import *

/usr/local/lib/python3.10/dist-packages/Cython/Distutils/old_build_ext.py:15: DeprecationWarning: dep_util is Deprecated. Use functions from setuptools instead.
  from distutils.dep_util import newer, newer_group


Compiling /usr/local/lib/python3.10/dist-packages/mujoco_py/cymj.pyx because it changed.
[1/1] Cythonizing /usr/local/lib/python3.10/dist-packages/mujoco_py/cymj.pyx


INFO:root:running build_ext
INFO:root:building 'mujoco_py.cymj' extension
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-310/usr/local/lib/python3.10/dist-packages/mujoco_py
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-310/usr/local/lib/python3.10/dist-packages/mujoco_py/gl
INFO:root:x86_64-linux-gnu-gcc -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/usr/local/lib/python3.10/dist-packages/mujoco_py -I/usr/local/lib/python3.10/dist-packages/mujoco_py/binaries/linux/mujoco210/include -I/usr/local/lib/python3.10/dist-packages/numpy/core/include -I/usr/include/python3.10 -c /usr/local/lib/python3.10/dist-packages/mujoco_py/cymj.c -o /usr/local/lib/python3.10/dist-packages/mujoco_py/g

In [6]:
n_episodes = 300000
print_every = 1000
device = 'cpu'
algorithm = 'PPO' # choices=['PPO']
env_id = 'CustomHopper-sudr-v0' # choices=['CustomHopper-source-v0', 'CustomHopper-target-v0', 'CustomHopper-sudr-v0']
save_path = './models/'
log_path = './logs/'

In [7]:
os.makedirs(save_path, exist_ok=True)
os.makedirs(log_path, exist_ok=True)

In [ ]:
# Parameters
num_iterations = 3  # Number of times to repeat the optimization and training process
best_mean_reward = -float('inf')
best_model = None

# Define save and log paths
save_base_path = './models/'
log_path = './logs/'
algorithm = 'ppo'
n_episodes = 100000  # Number of timesteps for training

# Ensure directories exist
os.makedirs(save_base_path, exist_ok=True)
os.makedirs(log_path, exist_ok=True)

# Load the human policy model
human = PPO.load("PPO_300k_target (2)")

for i in range(num_iterations):
    print(f"Iteration {i + 1}/{num_iterations}")

    # Step 1: Collect rewards from the target environment
    env_target = gym.make('CustomHopper-target-v0')
    real_actions, real_rewards = env_target.unwrapped.collect_real_data(human)

    # Step 2: Optimize parameters using CMA-ES
    env_source = gym.make('CustomHopper-source-v0')

    # Choose optimization method: CMA-ES
    optimized_params = env_source.unwrapped.cma_es_optimization(
        real_actions, real_rewards, n_iterations=10
    )
    print(f"Optimized Parameters (CMA-ES): {optimized_params}")

    # Step 3: Train policy using optimized parameters
    env_source.set_parameters(optimized_params)

    # Define save path for this iteration
    save_path = os.path.join(save_base_path, f'iteration_{i + 1}')
    os.makedirs(save_path, exist_ok=True)

    trained_model = PPO('MlpPolicy', env_source, verbose=1, tensorboard_log=log_path)

    checkpoint_callback = CheckpointCallback(
        save_freq=10000,
        save_path=save_path,
        name_prefix=f'rl_model_{i + 1}'
    )

    eval_callback = EvalCallback(
        env_source,
        best_model_save_path=save_path,
        log_path=log_path,
        eval_freq=5000,
        deterministic=True,
        render=False
    )

    trained_model.learn(total_timesteps=n_episodes, callback=[checkpoint_callback, eval_callback])
    trained_model.save(os.path.join(save_path, f"{algorithm}_final_model"))

    # Step 4: Evaluate the policy in the target environment
    print("Evaluating in target environment (target)...")
    mean_reward, std_reward = evaluate_policy(trained_model, env_target, n_eval_episodes=10)
    print(f"Target Environment - Mean Reward: {mean_reward}, Std Reward: {std_reward}")

    # Save the model if it's the best one
    if mean_reward > best_mean_reward:
        best_mean_reward = mean_reward
        best_model = trained_model
        best_model_path = os.path.join(save_base_path, f"ppo_custom_hopper_best_{i + 1}")
        best_model.save(best_model_path)
        print(f"New best model saved to {best_model_path}")

    # Close the environments after use
    env_source.close()
    env_target.close()

# Load and test the best model
if best_model is not None:
    best_model_file = f"ppo_custom_hopper_best_{i + 1}"
    best_model_path = os.path.join(save_base_path, best_model_file)
    best_model = PPO.load(best_model_path)
    env_target = gym.make('CustomHopper-target-v0')
    mean_reward, std_reward = evaluate_policy(best_model, env_target, n_eval_episodes=10)
    env_target.close()
    print(f"Best Model - Mean Reward: {mean_reward}, Std Reward: {std_reward}")
else:
    print("No best model was found.")